In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install pytimekr
!pip install sklego
!pip install neuralprophet[live]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pytimekr: filename=pytimekr-0.1.0-py3-none-any.whl size=7941 sha256=e9c54afed67d63a9e5632711611548c025c4a7b0050fd9b39a581d82ec6805dd
  Stored in directory: /root/.cache/pip/wheels/7c/54/31/d099bce210ce51c45eeb97e149699dedc66c78eb84c01be9c2
Successfully built pytimekr
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 224 kB 4.9 MB/s 
     |████████████████████████████████| 88 kB 7.9 MB/s 
     |████████████████████████████████| 1.1 MB 76.7 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=a415dbc7194a0af500443e5ff690df95f06b64ac296e2329eb1d54adc25b5e12
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl s

In [3]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
import numpy as np
import pandas as pd

### HOLIDAY
from pytimekr import pytimekr
### 제시된 시간만큼 로딩해주세요!
kr_holidays_2017 = pytimekr.holidays(2017)
kr_holidays_2018 = pytimekr.holidays(2018)
kr_holidays_2019 = pytimekr.holidays(2019)
kr_holidays_2020 = pytimekr.holidays(2020)
kr_holidays_2021 = pytimekr.holidays(2021)
kr_holidays_2022 = pytimekr.holidays(2022)

### TQDM
from tqdm.auto import tqdm

### VISUALIZATION
import seaborn as sns
import matplotlib.pyplot as plt

### SINCOS TRANSFORMATION
from sklearn.preprocessing import FunctionTransformer
from datetime import date

### RADIAN TRANSFORMATION
from sklego.preprocessing import RepeatingBasisFunction

### DECOMPOSING
from statsmodels.tsa.seasonal import seasonal_decompose

### plotting
import matplotlib.pyplot as plt
plt.rc('font', family = 'Malgun Gothic')
plt.rcParams['figure.figsize'] = (20, 20)

# warning 제거
import warnings
warnings.filterwarnings('ignore')

# datetime
from datetime import datetime

# scaler
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler

# Neural Prophet
from neuralprophet import NeuralProphet

# score
from sklearn.metrics import mean_squared_error

# Linear Regression
from sklearn.linear_model import LinearRegression


### Multioutput Regression
from sklearn.multioutput import RegressorChain

In [4]:
### HOLIDAYS

def make_holidays(df):
    holidays = []
    for i in tqdm(range(len(df))):
        #### 년도에 맞게 경우의 수를 추가해야 할 수 있습니다! 항상 년도에 맞춰주세요! n년치면 if문이 n개로 분기됩니다!
        if df["datetime"][i].date() in kr_holidays_2017:
            holidays.append(1)
        elif df["datetime"][i].date() in kr_holidays_2018:
            holidays.append(1)
        elif df["datetime"][i].date() in kr_holidays_2019:
            holidays.append(1)
        elif df["datetime"][i].date() in kr_holidays_2020:
            holidays.append(1)
        elif df["datetime"][i].date() in kr_holidays_2021:
            holidays.append(1)
        elif df["datetime"][i].date() in kr_holidays_2022:
            holidays.append(1)
        else:
            holidays.append(0)
    return holidays

### SINCOS TRANSFORMER

def sin_transformer(period):
	return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
	return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))
 
### Sequence

def make_dataset(data, label, window_size):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size*2):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size : i + window_size*2]))
    return np.array(feature_list), np.array(label_list)

### Dimension down
def dimension_down(df):
    dimension_df = np.empty((df.shape[0], df.shape[2]))
    for i in range(df.shape[0]):
        dimension_df[i] = df[i, (df.shape[1] - 1), :]
    return dimension_df

### Scaling
def scaling(df, scaler):
    for i in range(df.shape[0]):
        df[i, :, :] = scaler.transform(df[i, :, :])
    return df

In [5]:
### LOADING
df2017 = pd.read_csv("/content/gdrive/MyDrive/SeoulMoving/kaggle/test2017.csv", index_col = 0,sep = ",")
df2018 = pd.read_csv("/content/gdrive/MyDrive/SeoulMoving/kaggle/test2018.csv", index_col = 0,sep = ",")
df2019 = pd.read_csv("/content/gdrive/MyDrive/SeoulMoving/kaggle/test2019.csv", index_col = 0,sep = ",")
df2020 = pd.read_csv("/content/gdrive/MyDrive/SeoulMoving/kaggle/test2020.csv", index_col = 0,sep = ",")
df2021 = pd.read_csv("/content/gdrive/MyDrive/SeoulMoving/kaggle/test2021.csv", index_col = 0,sep = ",")
#df2022 = pd.read_csv("/content/gdrive/MyDrive/SeoulMoving/DATA/data2022.csv", index_col = 0,sep = ",")

### CONCAT
#data = pd.concat([df2017,df2018,df2019,df2020,df2021,df2022])
data = pd.concat([df2017,df2018,df2019,df2020,df2021])

### 필요한 열 추출
#data = data[["기준일ID","시간대구분","총생활인구수"]]

### datetime
data = data.reset_index()
data["기준일ID"] = data["기준일ID"].astype(str).apply(lambda x : x[0:4] + "-" + x[4:6] + "-" + x[6:8])
data["기준일ID"] = pd.to_datetime(data["기준일ID"])

### RENAME HOURS
data.rename(columns = {"시간대구분":"hour"},inplace = True)

### ADD DATETIME
data["datetime"] = data["기준일ID"] + pd.to_timedelta(data["hour"], unit = "h")

### 빈시간 정립
dates = pd.date_range(start = '2017-1-1 00:00:00', end = '2021-12-31 23:00:00' ,freq='h')

### 시간축 생성
time = pd.DataFrame()
time["datetime"] = dates

### merging
data = pd.merge(time, data, how = "left", on = "datetime")


################################################################################## CUSTIOMIZING 필요 ############################################################################################################
### 모든 year의 평균값으로 결측치 보간
year_list = [2017,2018,2019,2020,2021]

year_drop = int(data[data["hour"].isna()]["datetime"].dt.year.unique())

year_list.remove(year_drop)

interpolate_datetime = list(data[data["hour"].isna()]["datetime"])

interpolate_datetime_cluster = []
for year in year_list :
    interpolate_datetime_cluster.append([str(datetime).replace(str(year_drop), str(year)) for datetime in interpolate_datetime])


interpolate_elements = []
for cluster_datetime in interpolate_datetime_cluster :
    temp = pd.DataFrame()
    temp["datetime"] = cluster_datetime
    temp["datetime"] = pd.to_datetime(temp["datetime"])
    temp = pd.merge(temp, data, how = "left", on = "datetime")
    interpolate_elements.append(temp)

interpolate = pd.DataFrame()
for columns in interpolate_elements[0].columns :
    temp_values = 0
    if "인구" in columns :
        for dataframe in interpolate_elements :
            temp_values += dataframe[columns]
    temp_values = temp_values/len(year_list)
    interpolate[columns] = temp_values

interpolate["datetime"] = interpolate_datetime

### 원래 데이터 형식 맞춘후 보간된 데이터 저장
for columns in data.columns :
    if columns not in interpolate.columns :
        interpolate[columns] = np.nan

interpolate["기준일ID"] = interpolate["datetime"].dt.date
interpolate["hour"] = interpolate["datetime"].dt.hour
data.dropna(inplace = True)
data = pd.concat([data,interpolate])
data = data.sort_values(by = "datetime")
data.reset_index(drop = True, inplace = True)

################################################################################## CUSTIOMIZING 필요 ############################################################################################################

### month, day extraction
data["month"] = data["datetime"].dt.month
data["day"] = data["datetime"].dt.day

### MAKE holidays
data["holidays"] = make_holidays(data)

### DAY OF YEAR
data["day_of_year"] = data["datetime"].dt.day_of_year

### sincos transformation
data["month_sin"] = sin_transformer(12).fit_transform(data["month"])
data["month_cos"] = cos_transformer(12).fit_transform(data["month"])

data["day_sin"] = sin_transformer(365).fit_transform(data["day_of_year"])
data["day_cos"] = cos_transformer(365).fit_transform(data["day_of_year"])

### RADIAN FEATURES
rbf = RepeatingBasisFunction(n_periods=12,column="day_of_year",input_range=(1,365),remainder="drop")
rbf.fit(data)
radian_df = pd.DataFrame(index = data.index, data = rbf.transform(data))
radian_df.columns = ["radian" + "_" + str(i) for i in radian_df.columns]
data = pd.concat([data,radian_df],axis = 1)

### TIME SERIES DECOMPOSITION COLUMNS
mares_columns = []
for colname in data.columns :
    if "인구수" in colname :
        mares_columns.append(colname)
mares_columns.append("datetime")

### 필요없는 열 날리기
data.drop(columns = ["기준일ID"],inplace = True)

### 시계열분해를 위한 준비
decompose_data = data[mares_columns]
decompose_data.index = decompose_data["datetime"]
decompose_data.drop(columns = ["datetime"],inplace = True)

### residual extraction
### 24*30 is best
resid_decompose_data = pd.DataFrame()
for i in decompose_data.columns :
    result = seasonal_decompose(decompose_data[[i]], model = "additive", freq = 24*30,two_sided = False,extrapolate_trend = 'freq')
    resid_decompose_data[i] = pd.DataFrame(result.resid)['resid']
resid_decompose_data = resid_decompose_data.reset_index().dropna()
resid_decompose_data.reset_index(drop = True, inplace = True)
resid_decompose_data.columns = [i + "_r" for i in resid_decompose_data.columns]
resid_decompose_data.rename(columns = {"datetime_r":"datetime"}, inplace = True)

### trend extraction
trend_decompose_data = pd.DataFrame()
for i in decompose_data.columns :
    result = seasonal_decompose(decompose_data[[i]], model = "additive", freq = 24*30,two_sided = False,extrapolate_trend = 'freq')
    trend_decompose_data[i] = pd.DataFrame(result.trend)['trend']
trend_decompose_data = trend_decompose_data.reset_index().dropna()
trend_decompose_data.reset_index(drop = True, inplace = True)
trend_decompose_data.columns = [i + "_t" for i in trend_decompose_data.columns]
trend_decompose_data.rename(columns = {"datetime_t":"datetime"}, inplace = True)

### seasonal extraction
seasonal_decompose_data = pd.DataFrame()
for i in decompose_data.columns :
    result = seasonal_decompose(decompose_data[[i]], model = "additive", freq = 24*30,two_sided = False)
    seasonal_decompose_data[i] = pd.DataFrame(result.seasonal)['seasonal']
seasonal_decompose_data = seasonal_decompose_data.reset_index().dropna()
seasonal_decompose_data.reset_index(drop = True, inplace = True)
seasonal_decompose_data.columns = [i + "_s" for i in seasonal_decompose_data.columns]
seasonal_decompose_data.rename(columns = {"datetime_s":"datetime"}, inplace = True)

### merging timeseries decomposition

data = pd.merge(data,trend_decompose_data, how = "left", on = "datetime")
data = pd.merge(data,resid_decompose_data, how = "left", on = "datetime")
data = pd.merge(data,seasonal_decompose_data, how = "left", on = "datetime")

### MAKE SEQUENCE
#data = data.drop(columns = ["datetime","hour","month","day","day_of_year"])
#data.drop(columns = ["datetime","hour","index"],inplace = True)
#label = data["총생활인구수"]
#feature, label = make_dataset(data, label, window_size=1416)

### Robust Scaler
#scaler = RobustScaler().fit(dimension_down(feature))
#train_scaled_robust = scaling(feature, scaler)

### MinMax Scaler
#scaler = MinMaxScaler().fit(dimension_down(feature))
#train_scaled_minmax = scaling(feature, scaler)

### Standard Scaler
#scaler = StandardScaler().fit(dimension_down(feature))
#train_scaled_standard = scaling(feature, scaler)

#print(feature.shape)
#print(train_scaled_robust.shape)
#print(train_scaled_minmax.shape)
#print(train_scaled_standard.shape)
#print(label.shape)

  0%|          | 0/43824 [00:00<?, ?it/s]

In [6]:
data = data.drop(columns = ["index"])

In [7]:
shift_data = data["총생활인구수"].shift(-1416)

In [8]:
data["target"] = shift_data

In [9]:
test = data[data["target"].isna()]

In [10]:
train = data.dropna()

### CATBOOST GPU

In [11]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 28 kB/s 


In [12]:
import pandas as pd
import numpy as np

### label encoding
from sklearn.preprocessing import LabelEncoder

# catboostregressor
from catboost import CatBoostRegressor, Pool

# RMSE
from sklearn.metrics import mean_squared_error

# r2_score
from sklearn.metrics import r2_score

# fix random seed
import random
random.seed(2020)
random_seed = 2020

# directory
from glob import glob

# tqdm
from tqdm import tqdm

In [13]:
cart = ['hour','month','day','holidays','day_of_year']
for i in cart :
    train[i] = train[i].astype(int).astype("category")
    test[i] = test[i].astype(int).astype("category")

In [14]:
cart = ['hour','month','day','holidays','day_of_year']
train_x = train.drop(columns = ["datetime","target"])
train_y = train[["target"]]
test_x = test.drop(columns = ["datetime","target"])
# CatPool
pool_train = Pool(train_x, train_y, cat_features = cart)
pool_test = Pool(test_x, cat_features = cart)
# Modeling
model = CatBoostRegressor(iterations = 2000, loss_function = 'RMSE',task_type='GPU').fit(pool_train,verbose = 1)

Learning rate set to 0.043631
0:	learn: 3554.8701667	total: 27.3ms	remaining: 54.7s
1:	learn: 3443.8990253	total: 53.1ms	remaining: 53.1s
2:	learn: 3340.9873559	total: 79.1ms	remaining: 52.6s
3:	learn: 3244.4056934	total: 108ms	remaining: 54.1s
4:	learn: 3150.5559343	total: 135ms	remaining: 53.8s
5:	learn: 3060.5900160	total: 162ms	remaining: 53.7s
6:	learn: 2977.1207461	total: 189ms	remaining: 53.7s
7:	learn: 2896.5605405	total: 215ms	remaining: 53.6s
8:	learn: 2820.9362166	total: 240ms	remaining: 53.1s
9:	learn: 2751.5449714	total: 265ms	remaining: 52.7s
10:	learn: 2684.6690404	total: 289ms	remaining: 52.2s
11:	learn: 2619.2353021	total: 314ms	remaining: 52s
12:	learn: 2558.4244161	total: 340ms	remaining: 52s
13:	learn: 2499.3170728	total: 367ms	remaining: 52s
14:	learn: 2447.4141334	total: 392ms	remaining: 51.8s
15:	learn: 2395.2133969	total: 417ms	remaining: 51.8s
16:	learn: 2345.7823307	total: 443ms	remaining: 51.6s
17:	learn: 2300.5338954	total: 468ms	remaining: 51.5s
18:	learn: 

In [16]:
pred_y = model.predict(test_x)

In [17]:
submission = pd.read_csv("/content/gdrive/MyDrive/SeoulMoving/kaggle/sample_submission.csv")

In [19]:
submission["count"] = pred_y

In [20]:
submission.to_csv("/content/gdrive/MyDrive/SeoulMoving/catboost.csv",index = False)